The feature part is wrote for resampling. In this file, we extract file x_intermediate, which we computed in previous feature engineering part, resampling the imbalanced data using SMOTE(sythetic miniority oversampling technique)

In [1]:
import os
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
import random
from sklearn.model_selection import train_test_split

In [2]:
os.chdir('/home/luke/Desktop/braviant_datachallenge/')
x_use = pd.read_csv('x_intermediate.csv')
target = pd.read_csv('braviant.csv')['y']

In [3]:
x_use.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
x_use.shape

(10000, 9)

In [5]:
colnames = list(x_use.columns.values)

In [6]:
#Use SMOTE oversampling minority random 
sm = SMOTE(ratio = 'minority',random_state=666,kind ='svm')
x_res, y_res = sm.fit_sample(x_use, target)

In [7]:
pd.Series(y_res).value_counts()

0    9407
1    5228
dtype: int64

Now we have 5228 minority testcases. We then want to downsampling the majority to 6000

In [8]:
x_res = pd.DataFrame(x_res)

In [9]:
x_res.columns = colnames

As we noticed that the SMOTE use KNN based algorithm to compute similar sample, so for categorical variable, they would be float variable. Here we use round() function to make them categorical 

In [10]:
x_res['bin_negative'] = round(x_res['bin_negative'])
x_res['bin_positive'] = round(x_res['bin_positive'])

In [11]:
x_res.head()

,x0,x1,x2,x24,x25,x26,x27,bin_negative,bin_positive
0,166.0,1.0,-11.512925,1.0,5.455321,0.079545,14.0,1.0,0.0
1,818.0,0.0,0.693152,1.0,6.845880,0.000000,20.0,1.0,1.0
2,78.0,0.0,2.397896,1.0,4.317488,0.000000,8.0,2.0,1.0
3,272.0,0.0,1.098616,1.0,5.693732,0.000000,18.0,0.0,1.0
4,71.0,0.0,-11.512925,1.0,4.262680,0.000000,7.0,0.0,2.0


In [12]:
y_res = pd.Series(y_res)

In [13]:
df_for_split = pd.concat([x_res, y_res], axis=1)

In [14]:
collist = list(df_for_split.columns)
collist[-1] ='y'
df_for_split.columns = collist

In [15]:
major_df = df_for_split[df_for_split['y'] == 0]
minor_df = df_for_split[df_for_split['y'] == 1]

In [16]:
major_df.shape,minor_df.shape

((9407, 10), (5228, 10))

Downsampling the majority to 6000

In [17]:
rdm_major = major_df.sample(n=6000,replace = False)

In [27]:
rdm_major.shape,

(6000, 10)

In [28]:
major = rdm_major.drop(['y'],axis = 1)
minor = minor_df.drop(['y'],axis = 1)

y_major = rdm_major.y
y_minor = minor_df.y

In [29]:
major.shape,minor.shape,y_major.shape,y_minor.shape

((6000, 9), (5228, 9), (6000,), (5228,))

In [30]:
df_for_split_x = pd.concat([major, minor], axis=0)

In [31]:
df_for_split_y = pd.concat([y_major,y_minor],axis = 0)

In [32]:
df_for_split_y.shape,df_for_split_x.shape

((11228,), (11228, 9))

In [33]:
x_train, x_test, y_train, y_test = train_test_split(df_for_split_x, df_for_split_y, test_size=0.33, random_state=666)

In [34]:
x_train.to_csv('x_train.csv')
x_test.to_csv('x_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')